### <center>курс GeekBrains "Методы сбора и обработки данных из сети Интернет"<center>

### <center>Домашнее задание к уроку №4. "Парсинг HTML. XPath"<center>

1. Написать приложение, которое собирает основные новости с сайта на выбор news.mail.ru, lenta.ru, yandex-новости.  Для парсинга использовать XPath. Структура данных должна содержать:
    - название источника;
    - наименование новости;
    - ссылку на новость;
    - дата публикации


2. Сложить собранные новости в БД

**Подключение библиотек и скриптов**

In [1]:
import sys
import requests
from lxml import html
from pprint import pprint
from datetime import datetime, date
import datetime
import re
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError
from hashlib import md5

**Функция формирующая hash-строку из полученного словаря dict**

In [2]:
def id_hash(dict):
    '''Функция формирующая hash-строку из полученного словаря dict
    :param dict: полученного словаря dict
    :return result: hash-строка
    '''

    temp = str(dict).encode('utf-8')
    result = md5(temp).hexdigest()

    return result

**Функция записывающая данные из полученного списка словарей в mongo базу данных base**

In [3]:
def mongo_dict_insert(list, base):
    '''настоящая функция записывает данные из полученного списка словарей list в базу данных base
       важный момент: в составе словаря должен быть _id по которому настоящая функция определяет будет ли сделана запись,
       если запись с таким _id уже есть в base, то будет поднято exception и запись не произойдет
    :param list: список словарей
    :return [add_num, except_num]: сколько словарей [добавлено, уже есть в базе данных]
    '''
    
    add_num = 0
    except_num = 0
    
    for n in list:
        try:
            base.insert_one(n)
            add_num += 1
        except DuplicateKeyError:
            except_num += 1
            #print(f"Document with id = {n['_id']} already exists")

    print(f'на входе имеем список содержащий {len(list)} словарей')
    print(f'{add_num} словарей было добавлено в базу данных {base}')
    print(f'{except_num} словарей из предложенного списка уже есть в базе данных {base}')
            
    return [add_num, except_num]

**функция news_mail_ru_scraper**

In [4]:
def news_mail_ru_scraper(dom):    
    '''функция производит скрапинг dom в котором должны содержаться 5 заглавных фотоновостей 
       размещенных на сйте 'https://news.mail.ru/'       
    :param dom: 
    :return news_list: 
    '''

    news = dom.xpath('//a[contains(@class, "js-topnews__item")]')
     
    news_list = []
    i = 0
    
    for n in news:
        temp_dict = {}

        news_text = n.xpath('.//text()')[0].replace(u'\xa0', u' ')
        news_link = n.xpath('../a/@href')
        
        #чтобы добраться до даты и ссылки на первоисточник перейдем к странице с новостью и поработаем с ней
        i_headers = {'User-Agent': user_agent_list[i]} # ? возможно и не стоит менять агента на каждый новый запрос
        request = requests.get(news_link[0], headers=i_headers) # ? я таки сделал для перестраховки
        next_dom = html.fromstring(request.text)

        date = next_dom.xpath('//span/@datetime')
#         temp['date'] = date[0]
        date = datetime.datetime.strptime(str(date[0]), '%Y-%m-%dT%H:%M:%S%z')
        source_link = next_dom.xpath('//span[contains(@class, "note")]//@href')
                
        temp_dict['name'] = news_text
        temp_dict['date'] = date.strftime('%Y-%m-%d %H:%M')
        temp_dict['source_link'] = source_link[0]
        temp_dict['_id'] = id_hash(temp_dict) #добавляем в news_list хеш-строку построенную на предидущих данных для mongoid
        temp_dict['link'] = news_link[0]

        news_list.append(temp_dict)
        i += 1        
  
    return news_list

**функция news_lenta_scraper**

In [5]:
def news_lenta_scraper(dom):    
    '''функция производит скрапинг dom в котором должны содержаться 5 заглавных фотоновостей 
       размещенных на сйте 'https://lenta.ru//'  
    :param dom: 
    :return news_list: 
    '''

    news = dom.xpath('//a[contains(@class, "topnews")]')
     
    news_list = []
    i = 0
    
    for n in news:
        temp_dict = {}

        news_text = n.xpath('.//text()')[0]
        #print(news_text)
        if news_text == 'Все новости':
            break
        news_link = n.xpath('.//@href')[0]
        #print(news_link)
        news_time = n.xpath('.//time/text()')
        #print(news_time)
        
        if news_link[:20] == 'https://moslenta.ru/':            
            temp_date = re.search(r'\d\d\D\d\d\D\d\d\d\d', str(news_link))[0]
            temp_date_time = temp_date + ' ' + news_time[0]
            n_date = datetime.datetime.strptime(temp_date_time, '%m-%d-%Y %H:%M')
        
            source_link = news_link
        else:           
            temp_date = re.search(r'\d\d\d\d\D\d\d\D\d\d', str(news_link))[0]
            temp_date_time = temp_date + ' ' + news_time[0]
            n_date = datetime.datetime.strptime(temp_date_time, '%Y/%m/%d %H:%M')
            source_link = 'https://lenta.ru' + news_link
         
        temp_dict['name'] = news_text
        #temp_dict['date'] = temp_date
        temp_dict['date'] = n_date.strftime('%Y-%m-%d %H:%M')     
        temp_dict['_id'] = id_hash(temp_dict) #добавляем в news_list хеш-строку построенную на предидущих данных для mongo_id
        temp_dict['source_link'] = source_link
        #temp_dict['link'] = news_link[0]

        news_list.append(temp_dict)
        i += 1
  
    return news_list

**подключаем Mongo и работаем с базой данных 'news_db'**

In [6]:
client = MongoClient('127.0.0.1', 27017)
db = client['news_db']
news = db.news
#news.delete_many({})    # включаем эту строку если надо очистить базу данных 

**определем некоторые переменные**

In [7]:
my_header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'}

Список актуальных User agent по состоянию на 09.04.2022 для десктопных компьютеров взят из источника:
http://web-data-extractor.net/faq/spisok-aktualnyx-user-agent/

In [8]:
user_agent_list = ['Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.60 Safari/537.36',
                   'Mozilla/5.0 (Macintosh; Intel Mac OS X 12_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.60 Safari/537.36',
                   'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.60 Safari/537.36',
                   'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0)',
                   'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.1; Trident/4.0)',
                   'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0)',
                   'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)',
                   'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)',
                   'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; Trident/6.0)',
                   'Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko',
                   'Mozilla/5.0 (Windows NT 6.2; Trident/7.0; rv:11.0) like Gecko',
                   'Mozilla/5.0 (Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko',
                   'Mozilla/5.0 (Windows NT 10.0; Trident/7.0; rv:11.0) like Gecko',
                   'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0' 
                  ]

**формируем dom для 'https://news.mail.ru/'**

In [9]:
request = requests.get('https://news.mail.ru/', headers=my_header)
dom = html.fromstring(request.text)

**инициируем работу функции news_lenta_scraper, которая формирует news_mail_ru_list из dom**

In [10]:
news_mail_ru_list = news_mail_ru_scraper(dom)
#news_mail_ru_list

**инициируем работу функции mongo_dict_insert, которая записывает данные из news_mail_ru_list в 'news_db'**

In [11]:
mongo_dict_insert(news_mail_ru_list, news)

на входе имеем список содержащий 5 словарей
0 словарей было добавлено в базу данных Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'news_db'), 'news')
5 словарей из предложенного списка уже есть в базе данных Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'news_db'), 'news')


[0, 5]

**формируем dom для 'https://lenta.ru/'**

In [12]:
request = requests.get('https://lenta.ru/', headers=my_header)
dom = html.fromstring(request.text)

**инициируем работу функции news_lenta_scraper, которая формирует news_lenta_list из dom**

In [13]:
news_lenta_list = news_lenta_scraper(dom)
#news_lenta_list

**инициируем работу функции mongo_dict_insert, которая записывает данные из news_lenta_list в 'news_db'**

In [14]:
mongo_dict_insert(news_lenta_list, news)

на входе имеем список содержащий 13 словарей
0 словарей было добавлено в базу данных Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'news_db'), 'news')
13 словарей из предложенного списка уже есть в базе данных Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'news_db'), 'news')


[0, 13]

**не успеваю сделать код для https://yandex.ru/news/**

In [15]:
#news_yandex_link = 'https://yandex.ru/news/'